In [23]:
import pandas as pd
df = pd.read_csv("news1.csv")

In [ ]:
df.shape

In [48]:
df

,label,text_all
0,1,You Can Smell Hillary’s Fear Daniel Greenfield...
1,1,Watch The Exact Moment Paul Ryan Committed Pol...
2,0,Kerry to go to Paris in gesture of sympathy U....
3,1,Bernie supporters on Twitter erupt in anger ag...
4,0,The Battle of New York: Why This Primary Matte...
...,...,...
6330,0,State Department says it can't find emails fro...
6331,1,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,1,Anti-Trump Protesters Are Tools of the Oligarc...
6333,0,"In Ethiopia, Obama seeks progress on peace, se..."


In [25]:
#deal with tensors
import torch   

#handling text data
from torchtext import data   

#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

In [26]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

In [43]:
fields = [('label', LABEL),('text',TEXT)]

In [44]:
#loading custom dataset
training_data=data.TabularDataset(path = 'news1.csv',format = 'csv',fields = fields,skip_header = True)

In [45]:
#print preprocessed text
print(vars(training_data.examples[2]))

{'label': '0', 'text': ['Kerry', 'to', 'go', 'to', 'Paris', 'in', 'gesture', 'of', 'sympathy', 'U.S.', 'Secretary', 'of', 'State', 'John', 'F.', 'Kerry', 'said', 'Monday', 'that', 'he', 'will', 'stop', 'in', 'Paris', 'later', 'this', 'week', ',', 'amid', 'criticism', 'that', 'no', 'top', 'American', 'officials', 'attended', 'Sunday', '’s', 'unity', 'march', 'against', 'terrorism', '.', '\n\n', 'Kerry', 'said', 'he', 'expects', 'to', 'arrive', 'in', 'Paris', 'Thursday', 'evening', ',', 'as', 'he', 'heads', 'home', 'after', 'a', 'week', 'abroad', '.', 'He', 'said', 'he', 'will', 'fly', 'to', 'France', 'at', 'the', 'conclusion', 'of', 'a', 'series', 'of', 'meetings', 'scheduled', 'for', 'Thursday', 'in', 'Sofia', ',', 'Bulgaria', '.', 'He', 'plans', 'to', 'meet', 'the', 'next', 'day', 'with', 'Foreign', 'Minister', 'Laurent', 'Fabius', 'and', 'President', 'Francois', 'Hollande', ',', 'then', 'return', 'to', 'Washington', '.', '\n\n', 'The', 'visit', 'by', 'Kerry', ',', 'who', 'has', 'fami

In [52]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [53]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3 ,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)   

100%|██████████████████████████████████████████████████████████████████████▊| 398866/400000 [00:34<00:00, 12659.10it/s]

Size of TEXT vocabulary: 36958
Size of LABEL vocabulary: 2
[('the', 184488), (',', 180448), ('.', 152395), ('to', 98354), ('of', 91560), ('and', 80246), ('a', 73250), ('in', 64539), ('that', 47992), ('\n\n', 40991)]


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x000001B63D9D6D08>>, {'<unk>': 0, '<pad>': 1, 'the': 2, ',': 3, '.': 4, 'to': 5, 'of': 6, 'and': 7, 'a': 8, 'in': 9, 'that': 10, '\n\n': 11, 'is': 12, 'for': 13, '-': 14, '"': 15, 'on': 16, '\n': 17, '’s': 18, '“': 19, '”': 20, 'The': 21, 'it': 22, 'with': 23, 'as': 24, 'was': 25, 'are': 26, 'he': 27, 'be': 28, 'have': 29, 'by': 30, 'not': 31, 'Trump': 32, 'has': 33, 'I': 34, 'his': 35, 'said': 36, "'s": 37, 'from': 38, 'at': 39, 'this': 40, ':': 41, 'Clinton': 42, 'an': 43, 'who': 44, 'they': 45, 'will': 46, 'about': 47, 'or': 48, 'their': 49, 'you': 50, 'we': 51, 'would': 52, ')': 53, '(': 54, 'but': 55, 'more': 56, 'been': 57, '—': 58, 'people': 59, 'do': 60, 'all': 61, 'her': 62, 'were': 63, '?': 64, 'one': 65, 'had': 66, 'n’t': 67, 'which': 68, '\xa0': 69, 'out': 70, 'It': 71, 'what': 72, 'can': 73, 'she': 74, 'up': 75, 'But': 76, 'In': 77, 'Obama': 78, 'than': 79, 'also': 80, 'there': 81, 'our

100%|██████████████████████████████████████████████████████████████████████▊| 398866/400000 [00:50<00:00, 12659.10it/s]

In [54]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

In [55]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [56]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [57]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(36958, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 3,755,257 trainable parameters
torch.Size([36958, 100])


In [58]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [59]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [60]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [62]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

KeyboardInterrupt: 